# NYC Crimes, 2013 - 2023

## Data collection from NYC OpenData API
This section will only be run once.\
API source: https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i/about_data

In [29]:
# Import dependencies
import json
import pandas as pd
import requests
import time
import urllib.parse

In [2]:
# Define parameters for query
begin_date = '2013-01-01' # Enter the YYYY-MM-DD
end_date = '2023-12-31'
borough = 'MANHATTAN' #Choose one of the five boroughs

In [3]:
# String for query
query_string = f"boro_nm = '{borough}' AND rpt_dt >= '{begin_date}' AND rpt_dt <= '{end_date}'"

# Use urllib.parse.quote_plus to format the string for URL
safe_string = urllib.parse.quote_plus(query_string)

In [4]:
# Define variables for the While Loop. Create empty list to house the dictionaries created during the loop
records_count = 1
records = 0
offset = 0
set = 1
limit = 1000 # Per API documentation, can only pull 1000 records at a time
data_list = []

In [5]:
# While loop. While len of json_results > 0, query the URL.
while records_count != 0:
    # Define the URL for the JSON.
    url = f"https://data.cityofnewyork.us/resource/qgea-i56i.json?$where={safe_string}&$limit={limit}&$offset={offset}&$order=cmplnt_num"
    # Get the JSON
    json_results = requests.get(url).json()
    # Add to the number of results to track total number
    records += len(json_results)
    # Print message to console showing number of rows extracted, show set number.
    print(f"Extracting {len(json_results)} rows of data, Set # {set}")
    
    # Extract and store requested data into lists
    for x in range(len(json_results)):

        # Parse data into variables
        complaint_no = json_results[x]['cmplnt_num']
        reported_date = json_results[x]['rpt_dt']
        borough_name = json_results[x]['boro_nm']
        ofns = json_results[x]['ofns_desc']
        pd_desc = json_results[x]['pd_desc']
        law_cat_cd = json_results[x]['law_cat_cd']
        # Some records contain blank values in the lat/lon fields; as a result, the JSON for this [x] does not have 'latitude' or 'longitude' keys. Use try/except to store None into the variables for these records
        try:
            lat = json_results[x]['latitude']
        except:
            lat = None
        try:
            lon = json_results[x]['longitude']
        except: 
            lon = None

        # Append above data as a dictionary into data list
        data_list.append({
            "Complaint_No":complaint_no,
            "Reported_Dt":reported_date,
            "Borough_Name":borough_name,
            "Ofns_Desc": ofns,
            "PD_Desc":pd_desc,
            "Law_Cat": law_cat_cd,
            "Latitude": lat,
            "Longitude": lon
        })

    # Update the offset number by 1000
    offset = limit + offset
    # Update the set count
    set += 1
    # Update the records_count variable with count of json_results. Once it hits zero, loop will terminate
    records_count = len(json_results)
    # Delay of 1 second to avoid rate limiting
    time.sleep(1)
# Once loop condition is no longer true, print total number of records    
else: print(f"Total number of records extracted: {records}")

Extracting 1000 rows of data, Set # 1
Extracting 1000 rows of data, Set # 2
Extracting 1000 rows of data, Set # 3
Extracting 1000 rows of data, Set # 4
Extracting 1000 rows of data, Set # 5
Extracting 1000 rows of data, Set # 6
Extracting 1000 rows of data, Set # 7
Extracting 1000 rows of data, Set # 8
Extracting 1000 rows of data, Set # 9
Extracting 1000 rows of data, Set # 10
Extracting 1000 rows of data, Set # 11
Extracting 1000 rows of data, Set # 12
Extracting 1000 rows of data, Set # 13
Extracting 1000 rows of data, Set # 14
Extracting 1000 rows of data, Set # 15
Extracting 1000 rows of data, Set # 16
Extracting 1000 rows of data, Set # 17
Extracting 1000 rows of data, Set # 18
Extracting 1000 rows of data, Set # 19
Extracting 1000 rows of data, Set # 20
Extracting 1000 rows of data, Set # 21
Extracting 1000 rows of data, Set # 22
Extracting 1000 rows of data, Set # 23
Extracting 1000 rows of data, Set # 24
Extracting 1000 rows of data, Set # 25
Extracting 1000 rows of data, Set 

In [6]:
# Create DataFrame
crime_df = pd.DataFrame(data_list)

In [7]:
# DataFrame info
crime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1280579 entries, 0 to 1280578
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   Complaint_No  1280579 non-null  object
 1   Reported_Dt   1280579 non-null  object
 2   Borough_Name  1280579 non-null  object
 3   Ofns_Desc     1280579 non-null  object
 4   PD_Desc       1280579 non-null  object
 5   Law_Cat       1280579 non-null  object
 6   Latitude      1280576 non-null  object
 7   Longitude     1280576 non-null  object
dtypes: object(8)
memory usage: 78.2+ MB


In [ ]:
# Drop rows with null values
cleaned_crime_df = crime_df.dropna()

In [61]:
# Preview DataFrame
cleaned_crime_df.head(10)

,Complaint_No,Reported_Dt,Borough_Name,Ofns_Desc,PD_Desc,Law_Cat,Latitude,Longitude
1139070,88397298,2013-01-01,MANHATTAN,VEHICLE AND TRAFFIC LAWS,LEAVING SCENE-ACCIDENT-PERSONA,MISDEMEANOR,40.84812006,-73.93088385
1138649,88378509,2013-01-01,MANHATTAN,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,MISDEMEANOR,40.784781,-73.977134
1138648,88378508,2013-01-01,MANHATTAN,INTOXICATED & IMPAIRED DRIVING,"INTOXICATED DRIVING,ALCOHOL",MISDEMEANOR,40.77220939,-73.98217925
1138647,88378506,2013-01-01,MANHATTAN,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,MISDEMEANOR,40.788597,-73.977179
1138646,88378505,2013-01-01,MANHATTAN,PETIT LARCENY,"LARCENY,PETIT FROM BUILDING,UN",MISDEMEANOR,40.788597,-73.977179
1139066,88397294,2013-01-01,MANHATTAN,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,MISDEMEANOR,40.86510713,-73.91844727
1139069,88397297,2013-01-01,MANHATTAN,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,MISDEMEANOR,40.850452,-73.938469
1139073,88397307,2013-01-01,MANHATTAN,VEHICLE AND TRAFFIC LAWS,LEAVING SCENE-ACCIDENT-PERSONA,MISDEMEANOR,40.86446248,-73.91894337
1139082,88400523,2013-01-01,MANHATTAN,OFFENSES AGAINST PUBLIC ADMINI,RESISTING ARREST,MISDEMEANOR,40.79993102,-73.94066426
1138650,88378511,2013-01-01,MANHATTAN,PETIT LARCENY,"LARCENY,PETIT FROM AUTO",MISDEMEANOR,40.785443,-73.977717


In [57]:
# Save cleaned_crime_df to CSV file
cleaned_crime_df.to_csv('Output/NPYD_Complaints_01-01-13_12-31-23.csv', index=False)

## Drop `PD_CD` Field
Per NYC Open Data Data Dictionary:
 - **PD_CD**: Three digit internal classification code (more granular than Key Code)
 - **OFNS_DESC**: Description of offense corresponding with key code

There are 392 unique strings in `PD_Desc` whereas `Ofns_Desc` has 67.\
`Ofns_Desc` will be sufficient for this project. `PD_CD` will be omitted. The resulting .csv file is ~40 MiB smaller.

In [37]:
# Read original CSV file
crime_df = pd.read_csv('Output/NPYD_Complaints_01-01-13_12-31-23.csv')

In [38]:
# Check unique values in `PD_CD` and `OFNS_DESC`
print(len(crime_df['PD_Desc'].unique().tolist()))
print(len(crime_df['Ofns_Desc'].unique().tolist()))

392
67


In [40]:
# Drop `PD_CD` from DataFrame. Preview.
crime_df_update = crime_df[['Complaint_No', 'Reported_Dt', 'Borough_Name','Ofns_Desc','Law_Cat','Latitude','Longitude']]
crime_df_update.head()

,Complaint_No,Reported_Dt,Borough_Name,Ofns_Desc,Law_Cat,Latitude,Longitude
0,109140080,2014-03-21,MANHATTAN,GRAND LARCENY,FELONY,40.721109,-74.009995
1,109140082,2014-03-21,MANHATTAN,GRAND LARCENY,FELONY,40.724833,-73.997278
2,109140084,2014-03-23,MANHATTAN,HARRASSMENT 2,VIOLATION,40.719914,-73.997070
3,109140085,2014-03-23,MANHATTAN,PETIT LARCENY,MISDEMEANOR,40.724833,-73.997278
4,109140087,2014-03-23,MANHATTAN,FORGERY,FELONY,40.720656,-73.988061


In [42]:
# Export to CSV
crime_df_update.to_csv('Output/NPYD_Complaints_01-01-13_12-31-23_v2.csv', index=False)